In [4]:
# Load all packages necessary for analysis
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import json
import re
import os
import timeit
import filecmp
import url_stripper

# deck_type_urls

## Gather

In [5]:
# make a webcrawler to gather the data from mtgtop8.com on competitive decks
url = 'https://www.mtgtop8.com/format?f=ST'
url_standard_page_response = requests.get(url)

In [6]:
# save the response
with open('C:\\Users\\muroc\\Documents\\MTG\\data\\url_standard_page_response.html', mode='wb') as file:
    file.write(url_standard_page_response.content)

In [7]:
# open response as a BeautifulSoup object
with open('C:\\Users\\muroc\\Documents\\MTG\\data\\url_standard_page_response.html') as file:
    soup = BeautifulSoup(file, 'lxml')

In [8]:
url = {'url': soup.find_all(href=re.compile(r"archetype\?a"))}
deck_type_urls = pd.DataFrame(data=url)

In [9]:
# create column for deck type and clean the column
url_stripper.url_stripper(deck_type_urls, 'url', 'type', '(>.+<)')

In [10]:
deck_type_urls

,url,type
0,"<a href=""archetype?a=832&amp;meta=187&amp;f=ST...",Simic Aggro
1,"<a href=""archetype?a=849&amp;meta=187&amp;f=ST...",Adventure
2,"<a href=""archetype?a=286&amp;meta=187&amp;f=ST...",Rakdos Aggro
3,"<a href=""archetype?a=12&amp;meta=187&amp;f=ST""...",Red Deck Wins
4,"<a href=""archetype?a=229&amp;meta=187&amp;f=ST...",Gruul Aggro
5,"<a href=""archetype?a=292&amp;meta=187&amp;f=ST...",Golgari Aggro
6,"<a href=""archetype?a=349&amp;meta=187&amp;f=ST...",Mono Black Aggro
7,"<a href=""archetype?a=150&amp;meta=187&amp;f=ST...",Selesnya Aggro
8,"<a href=""archetype?a=323&amp;meta=187&amp;f=ST...",Bant Aggro
9,"<a href=""archetype?a=386&amp;meta=187&amp;f=ST...",Temur Aggro


In [11]:
# extract url from the url column
deck_type_urls['url'] = deck_type_urls.url.str.extract('(archetype.+f\=ST)')

In [12]:
# add root url to the data
urls = []
types = []

for i in np.arange(deck_type_urls.shape[0]):
    urls.append(str('https://www.mtgtop8.com/') + deck_type_urls.url[i])
    types.append(deck_type_urls.type[i] + str('.html'))

In [13]:
deck_type_urls['url'] = urls
deck_type_urls['type'] = types
deck_type_urls['type'] = deck_type_urls.type.str.replace(' ', '_')
deck_type_urls['type'] = deck_type_urls.type.str.replace('/', '-')

In [15]:
# create web crawler to request html pages from deck_type_urls

# create new path and name it newpath
newpath = 'C:\\Users\\muroc\\Documents\\MTG\\type_html_files'

# check to make sure that there is no path that matches newpath and create a folder called html_files if there isn't
if not os.path.exists(newpath):
    os.makedirs(newpath)

# change current directory to newpath
os.chdir(newpath)

# scrape all html files from urls in deck_type_urls and put them in html_files folder
for i in np.arange(deck_type_urls.shape[0]):
    type_html = requests.get(deck_type_urls.url[i])
    with open(deck_type_urls.type[i], 'wb') as file:
        file.write(type_html.content)